# Gai/Gen: Text-to-Text (TTT)

## 1.1 Setting Up

We will create a seperate virtual environment for this to avoid conflicting dependencies that each underlying model requires.

```sh
sudo apt update -y && sudo apt install ffmpeg git git-lfs -y
conda create -n TTT python=3.10.10 -y
conda activate TTT
pip install -e ".[TTT]"
```

The following examples has been tested on the following environment:

-   NVidia GeForce RTX 2060 6GB
-   Windows 11 + WSL2
-   Ubuntu 22.04
-   Python 3.10
-   CUDA Toolkit 11.8
-   openai 1.6.1
-   anthropic 0.8.1
-   transformers 4.36.2
-   bitsandbytes 0.41.3.post2
-   scipy 1.11.4
-   accelerate 0.25.0
-   llama-cpp-python 0.2.25


## 1.2 Running as a Library

### OpenAI GPT4

For (1) and (2) below, you will use the GaiGen library to call OpenAI's GPT4.
You will need to get an API key from OpenAI. 
Create .env file in project root directory and insert the OpenAI API Key below:

```sh
OPENAI_API_KEY=<your key here>
```

In [ ]:
### 1. GPT4 Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('gpt-4')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}])
print(response.choices[0].message.content)

In [ ]:
### 2. GPT4 Text-to-Text Streaming

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('gpt-4')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],stream=True)
for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content,end='',flush=True)

### Mistral 7B 8k-context 4-bit quantized

For (3) and (4), you will run Mistral 7B locally. Clone TheBloke's 4-bit quantized version of Mistral-7B model from hugging face. This model utilizes the exLlama loader for increased performance. Make sure you have huggingface-hub installed, if not run `pip install huggingface-hub`.

In [ ]:
%%bash
huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GPTQ \
        --local-dir ~/gai/models/Mistral-7B-Instruct-v0.1-GPTQ \
        --local-dir-use-symlinks False

In [ ]:
### 3. Mistral Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b-exllama')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100, stream=False)
print(response.choices[0].message.content)


In [ ]:
### 4. Mistral Text-to-Text Streaming

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b-exllama')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100,stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content,end='',flush=True)


### Yarn-Mistral-7B 128k-context 4-bit quantized

Repeat the earlier examples but using a different version of Mistral-7B model with a larger context window.

In [ ]:
%%bash
huggingface-cli download TheBloke/Yarn-Mistral-7B-128k-GPTQ \
        --local-dir ~/gai/models/Yarn-Mistral-7B-128k-GPTQ \
        --local-dir-use-symlinks False

According to their paper, the perplexity seems better than the original once the token length is greater than 10k.

![perplexity-of-mistral7b-128k](https://raw.githubusercontent.com/jquesnelle/yarn/mistral/data/proofpile-long-small-mistral.csv.png)



In [1]:
### 3. Mistral Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b_128k-exllama')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100)
print(response.choices[0].message.content,end='',flush=True)

GENERATING:
Once upon a time, there was a little girl who loved to play outside in the sunshine. One day she went out into her backyard and saw something strange – it looked like a giant spider web! She walked closer and realized that it wasn’t a spider at all but rather an intricate pattern of leaves on the ground. The more she explored this beautiful design, the happier she became until finally she sat down under its shade for hours enjoying nature’s beauty around her

In [2]:
### 4. Mistral Text-to-Text Streaming

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b_128k-exllama')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100,stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content,end='',flush=True)

STREAMING:
 Once upon a time, there was a little girl who loved to play outside in the sunshine. One day she went out into her backyard and saw something strange – it looked like a giant spider web! She walked closer and realized that it wasn’t a spider at all but rather an intricate pattern of leaves on the ground. The more she explored this beautiful design, the happier she became until finally she sat down under its shade for hours enjoying nature’s beauty around her

### Anthropics Claude2.1

The following example uses Anthropics Claude2.1 100k context window size model. Get API Key from Anthropics and add it to the .env file.
```sh
ANTHROPIC_APIKEY=<your key here>
```

In [ ]:
### 5. Claude-2.1 Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('claude2-100k')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_tokens_to_sample=100)
print(response.choices[0].message.content,end='',flush=True)

In [ ]:
### 6. Claude-2.1 Text-to-Text Streaming

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('claude2-100k')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_tokens_to_sample=100,stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content,end='',flush=True)

### Llama2 7B with HuggingFace transformers

Follow the instructions [here](https://huggingface.co/docs/transformers/main/en/model_doc/llama2) to signup with Meta to download the LLaMa-2 model.
Download the model in HuggingFace format from [here] (https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) into ~/gai/models/Llama-2-7b-chat-hf.

In [ ]:
### 7. Llama2-7B Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
from IPython.utils import io
with io.capture_output() as captured:
    gen = Gaigen.GetInstance().load('llama2-transformers')
    response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100)
print(response.choices[0].message.content,end='',flush=True)

In [ ]:
### 8. Llama2-7B Text-to-Text Streaming

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('llama2-transformers')
from IPython.utils import io
with io.capture_output() as captured:
    response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100,stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content,end='',flush=True)

### Llama2 7B GGUF with LlaMaCPP (CPU only)

The following example uses GGUF formatted version of Mistral-7B for LlaMaCPP. This can be used when you want the model to run off CPU only

In [ ]:
%%bash
# Download the model
huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF \
                mistral-7b-instruct-v0.1.Q4_K_M.gguf  \
                config.json \
                --local-dir ~/gai/models/Mistral-7B-Instruct-v0.1-GGUF \
                --local-dir-use-symlinks False

In [ ]:
## 9. Mistral-7B CPU-Only Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b-llamacpp')
from IPython.utils import io
import sys
with io.capture_output() as captured:
    # Redirect stderr to stdout
    sys.stderr = sys.stdout    
    response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100)
print(response.choices[0].message.content,end='',flush=True)

In [ ]:
## 10. Mistral-7B CPU-Only Text-to-Text Generation

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b-llamacpp')
from IPython.utils import io
with io.capture_output() as captured:
    response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100,stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content,end='',flush=True)

## 1.3 Running as a Service

### Step 1: Start Docker container

In [ ]:
%%bash

# Stop any container with the same name
docker rm -f gai-ttt

# Start the container
docker run -d \
    --name gai-ttt \
    -p 12031:12031 \
    --gpus all \
    -v ~/gai/models:/app/models \
    kakkoii1337/gai-ttt:latest

# Wait for model to load
sleep 30

# Confirm its running
docker logs gai-ttt

When the loading is completed, the logs should show this:

```bash
INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:12031 (Press CTRL+C to quit)
```

### Step 2: Test using Mistral7B-8k

The default model is Mistral7B-8k context size. You can test it by running the following command:

In [ ]:
%%bash
#### Step 2: Test

curl -X POST \
    http://localhost:12031/gen/v1/chat/completions \
    -H 'Content-Type: application/json' \
    -s \
    -N \
    -d "{\"model\":\"mistral7b-exllama\", \
        \"messages\": [ \
            {\"role\": \"user\",\"content\": \"Tell me a story\"}, \
            {\"role\": \"assistant\",\"content\": \"\"} \
        ],\
        \"max_new_tokens\":100, \
        \"stream\":true}"


### Step 3: Change the default model to Mistral7B-128k

Change the default model to Mistral7b-128k by changing the config file in the container.

In [ ]:
%%bash

# Change default model
curl -X PUT \
    http://localhost:12031/gen/v1/chat/config \
    -H 'Content-Type: application/json' \
    -s \
    -d '{"generator_name":"mistral7b_128k-exllama",                       
        "generator_config": {                               
            "type": "ttt",
            "model_name": "Mistral7B_128k-ExLlama",
            "engine": "ExLlamaV2_TTT",
            "model_path": "models/Yarn-Mistral-7B-128k-GPTQ",
            "model_basename": "model",
            "max_seq_len": 65536,
            "stopping_words": [],
            "hyperparameters": {
                "temperature": 1.2,
                "top_p": 0.15,
                "min_p": 0.0,
                "top_k": 50,
                "max_new_tokens": 1000,
                "typical": 0.0,
                "token_repetition_penalty_max": 1.25,
                "token_repetition_penalty_sustain": 256,
                "token_repetition_penalty_decay": 128,
                "beams": 1,
                "beam_length": 1
            }
        }
    }'



restart the container

```bash
# Restart container
docker stop gai-ttt
docker start gai-ttt
```

Wait for the model to load and check the logs to confirm that the new model is loaded.

```bash
docker logs gai-ttt
```

### Step 4. Test Again using Mistral7B-128k

Then test again with mistral7b-128k model.

In [ ]:
%%bash
echo STREAMING
curl -X POST \
    http://localhost:12031/gen/v1/chat/completions \
    -H 'Content-Type: application/json' \
    -s \
    -N \
    -d "{\"model\":\"mistral7b_128k-exllama\", \
        \"messages\": [ \
            {\"role\": \"user\",\"content\": \"Tell me a story\"}, \
            {\"role\": \"assistant\",\"content\": \"\"} \
        ],\
        \"stream\":true}" \
        | python ../print_delta.py
echo
echo NON-STREAMING
curl -X POST \
    http://localhost:12031/gen/v1/chat/completions \
    -H 'Content-Type: application/json' \
    -s \
    -N \
    -d "{\"model\":\"mistral7b_128k-exllama\", \
        \"messages\": [ \
            {\"role\": \"user\",\"content\": \"Tell me a story\"}, \
            {\"role\": \"assistant\",\"content\": \"\"} \
        ]}"
